Kaggle Intermediate Machine Learning Tutorial

## Importing Libraries

In [100]:
import numpy as np 
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error # MAE metric
from sklearn.model_selection import train_test_split

### Random Forest Regressor MAE Scoring of test data

In [101]:
def score_dataset(X_train, X_valid, y_train, y_valid): 
    model = RandomForestRegressor(n_estimators=100)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Feature Selection
Only using numerical features to keep things simple

In [102]:
df = pd.read_csv('melb_data.csv')

# Target is house price
y = df.Price

# Use only numerical predictors
melb_predictors = df.drop(['Price'], axis=1) # get rid of target
X = melb_predictors.select_dtypes(exclude=['object']) # drop categorial features

X_train, X_test, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

### Dataset has some missing values
We can either drop them or impute between the missing values

In [103]:
df = pd.read_csv('melb_data.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


## MAE from Dropping Missing Numerical Data

In [104]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()] # these columns have missing data

reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_test.drop(cols_with_missing, axis=1)

print(f'MAE from dropping: {score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)}')

MAE from dropping: 188749.31980278538


## Simple Imputation

In [105]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.fit_transform(X_test))

# Imputation removes column names, so we have to add them back in
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_test.columns

# Score this method
print(f'MAE from imputation: {score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)}')

MAE from imputation: 191998.98715162353


### What happens if we keep track of which values were imputed?

In [106]:
# Copy the original data
X_train_plus = X_train.copy()
X_test_plus = X_test.copy()

# Make new columns to keep track of imputations
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_test_plus[col + '_was_missing'] = X_test_plus[col].isnull()

# Simple imputaiton
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_test_plus = pd.DataFrame(my_imputer.fit_transform(X_test_plus))

# Re-add column names
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_test_plus.columns = X_test_plus.columns

# Score this method
print(f'MAE of imputation + keeping track of columns: {score_dataset(imputed_X_train_plus, imputed_X_test_plus, y_train, y_valid)}')

MAE of imputation + keeping track of columns: 195045.24179553264


## Handling Categorical Features

In [121]:
X = melb_predictors.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=.2)

s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print(f'Categorical features: {object_cols}')

Categorical features: ['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea', 'Regionname']


### Dropping Categorical Features

In [131]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_test = X_test.select_dtypes(exclude=['object'])

# Drop missing values as well
cols_with_missing = [col for col in drop_X_train.columns if X_train[col].isnull().any()] # these columns have missing data

reduced_X_train = drop_X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = drop_X_test.drop(cols_with_missing, axis=1)

# Score dataset
print(f'MAE from dropping: {score_dataset(reduced_X_train, reduced_X_valid, y_train, y_test)}')

MAE from dropping: 186259.67136988972
